In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/jester_dataset.zip .

In [4]:
!unzip jester_dataset.zip

Archive:  jester_dataset.zip
   creating: jester_dataset/
  inflating: jester_dataset/jester_X_train.npy  
  inflating: jester_dataset/jester_y_train.npy  
  inflating: jester_dataset/jester_X_valid.npy  
  inflating: jester_dataset/jester_y_valid.npy  


In [0]:
import numpy as np

X_train = np.load('jester_dataset/jester_X_train.npy')
y_train = np.load('jester_dataset/jester_y_train.npy')
X_valid = np.load('jester_dataset/jester_X_valid.npy')
y_valid = np.load('jester_dataset/jester_y_valid.npy')

In [0]:
X_train_fp= X_train.astype(np.float16)
y_train_int= y_train.astype(np.uint8)
X_valid_fp= X_valid.astype(np.float16)
y_valid_int= y_valid.astype(np.uint8)

In [0]:
del X_train
del y_train
del X_valid
del y_valid

In [9]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt


In [0]:
import os
import math
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()
scaled_images_train  = np.array(scaler.fit_transform(X_train_fp.reshape(-1, 30*64*64)))
scaled_images_train  = np.array(scaled_images_train.reshape(-1, 30, 64, 64, 1))


In [0]:
scaler = StandardScaler()
scaled_images_valid  = np.array(scaler.fit_transform(X_valid_fp.reshape(-1, 30*64*64)))
scaled_images_valid  = np.array(scaled_images_valid.reshape(-1, 30, 64, 64, 1))

In [0]:
del X_train_fp
del X_valid_fp


In [0]:
# My model
class Conv3DModel(tf.keras.Model):
  def __init__(self):
    super(Conv3DModel, self).__init__()
    # Convolutions
    self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
    self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')
   
    # LSTM & Flatten
    self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
    self.flatten =  tf.keras.layers.Flatten(name="flatten")

    # Dense layers
    self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
    self.out = tf.keras.layers.Dense(6, activation='softmax', name="output")
    

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.convLSTM(x)
    #x = self.pool2(x)
    #x = self.conv3(x)
    #x = self.pool3(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.out(x)

In [0]:
model = Conv3DModel()

In [0]:
# choose the loss and optimizer methods
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [0]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_today/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True)


In [18]:
# Run the training 
history = model.fit(scaled_images_train, y_train_int,
                    callbacks = [cp_callback],
                    validation_data=(scaled_images_valid, y_valid_int),
                    batch_size=32,
                    epochs=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 25876 samples, validate on 3179 samples
Epoch 1/10
25856/25876 [============================>.] - ETA: 0s - loss: 0.6322 - acc: 0.7678
Epoch 00001: saving model to training_today/cp-0001.ckpt
25876/25876 [==============================] - 92s 4ms/sample - loss: 0.6321 - acc: 0.7679 - val_loss: 0.3442 - val_acc: 0.8849
Epoch 2/10
25856/25876 [============================>.] - ETA: 0s - loss: 0.3466 - acc: 0.8819
Epoch 00002: saving model to training_today/cp-0002.ckpt
25876/25876 [==============================] - 85s 3ms/sample - loss: 0.3469 - acc: 0.8819 - val_loss: 0.2560 - val_acc: 0.9166
Epoch 3/10
25856/25876 [============================>.] - ETA: 0s - loss: 0.2253 - acc: 0.9253
Epoch 00003: saving model to training_today/cp-0003.ckpt
25876/25876 [==============================] - 84s 3ms/sample - 

In [19]:
history.history


{'acc': [0.7679317,
  0.88185966,
  0.92529756,
  0.9479054,
  0.9648709,
  0.97399133,
  0.98492813,
  0.9847735,
  0.9882517,
  0.9913047],
 'loss': [0.6321346970057078,
  0.34694595218239177,
  0.2252556763268048,
  0.15839191958190743,
  0.1039202843096944,
  0.07406272675567357,
  0.04573429997320507,
  0.044083863178289384,
  0.034707821390872404,
  0.02596139433367995],
 'val_acc': [0.88486946,
  0.91664046,
  0.93331236,
  0.943693,
  0.9399182,
  0.9399182,
  0.9396036,
  0.95281535,
  0.9480969,
  0.95250076],
 'val_loss': [0.3442333872896127,
  0.25604453895237506,
  0.20794550315853408,
  0.180431334266967,
  0.2165728809043048,
  0.22881977231285414,
  0.2520445537580263,
  0.2324329875775804,
  0.23407297408796993,
  0.24750294937485154]}

In [0]:
# save the model for use in the application
model.save_weights('weights/3d-cnn-basic')

In [23]:
!ls weights

3d-cnn-basic.data-00000-of-00002  3d-cnn-basic.index
3d-cnn-basic.data-00001-of-00002  checkpoint


In [28]:
!zip -r weights.zip weights

  adding: weights/ (stored 0%)
  adding: weights/3d-cnn-basic.data-00000-of-00002 (deflated 8%)
  adding: weights/checkpoint (deflated 38%)
  adding: weights/3d-cnn-basic.data-00001-of-00002 (deflated 81%)
  adding: weights/3d-cnn-basic.index (deflated 66%)


In [0]:
!cp weights.zip /content/drive/My\ Drive/ 

In [0]:

class Conv3DModel(tf.keras.Model):
  def __init__(self):
    super(Conv3DModel, self).__init__()
    # Convolutions
    self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
    self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')
   
    # LSTM & Flatten
    self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
    self.flatten =  tf.keras.layers.Flatten(name="flatten")

    # Dense layers
    self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
    self.out = tf.keras.layers.Dense(6, activation='softmax', name="output")
    

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.convLSTM(x)
    #x = self.pool2(x)
    #x = self.conv3(x)
    #x = self.pool3(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.out(x)

In [0]:
model = Conv3DModel()

In [0]:
# use tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((scaled_images_train, y_train_fp))
cv_dataset = tf.data.Dataset.from_tensor_slices((scaled_images_valid, y_valid_fp))

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [0]:
@tf.function
def train_step(image, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(image)
        # Get the error/loss on these predictions
        loss = loss_fn(targets, predictions)
    # Compute the gradient which respect to the loss
    grads = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

In [0]:
@tf.function
def valid_step(image, targets):
    predictions = model(image)
    t_loss = loss_fn(targets, predictions)
    # Set the metrics for the test
    valid_loss(t_loss)
    valid_accuracy(targets, predictions)

In [0]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, 'training_checkpoints/tf_ckpts', max_to_keep=10)
ckpt.restore(manager.latest_checkpoint)

In [0]:
epoch = 10
batch_size = 32
b = 0
training_acc = []
validation_acc = []
for epoch in range(epoch):
    # Training set
    for images_batch, targets_batch in train_dataset.batch(batch_size):
        train_step(images_batch, targets_batch)
        template = '\r Batch {}/{}, Loss: {}, Accuracy: {}'
        print(template.format(
            b, len(training_targets), train_loss.result(), 
            train_accuracy.result()*100
        ), end="")
        b += batch_size
    # Validation set
    for images_batch, targets_batch in cv_dataset.batch(batch_size):
        valid_step(images_batch, targets_batch)

    template = '\nEpoch {}, Valid Loss: {}, Valid Accuracy: {}'
    print(template.format(
        epoch+1,
        valid_loss.result(), 
        valid_accuracy.result()*100)
    )
    training_acc.append(float(train_accuracy.result()*100))
    validation_acc.append(float(valid_accuracy.result()*100))
    ckpt.step.assign_add(1)
    save_path = manager.save()
    print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    train_accuracy.reset_states()
    train_loss.reset_states()